In [2]:
import os
from os.path import dirname
root_path = dirname(dirname(os.getcwd()))
print(root_path)
import sys
sys.path.append(root_path + '/RemainingCycleTimePrediction/2_Scripts/')
import pandas as pd
import numpy as np
import time, datetime
import pickle as pkl
import itertools
from sklearn.preprocessing import OneHotEncoder

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split

from Event_log_processing_utils import Extract_trace_and_temporal_features, Extract_prefix

import warnings
warnings.filterwarnings("ignore")

data_dir = root_path + '/RemainingCycleTimePrediction/1_Data/'
project_dir = root_path + '/RemainingCycleTimePrediction/'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

/home/ec2-user
/home/ec2-user
cuda:0


In [4]:
data_name = 'BPIC20'

In [6]:
tab_all = pd.read_csv(data_dir+data_name+"_processed_all.csv")
tab_train= pd.read_csv(data_dir+data_name+"_processed_train.csv")
tab_valid = pd.read_csv(data_dir+data_name+"_processed_valid.csv")
tab_test = pd.read_csv(data_dir+data_name+"_processed_test.csv")
tab_all.head()

,Case_ID,Activity,timestamp
0,declaration 100000,Declaration SUBMITTED by EMPLOYEE,2018/01/30 08:20:07
1,declaration 100000,Declaration APPROVED by ADMINISTRATION,2018/02/07 08:58:46
2,declaration 100000,Declaration FINAL_APPROVED by SUPERVISOR,2018/02/08 09:59:05
3,declaration 100000,Request Payment,2018/02/09 11:42:49
4,declaration 100000,Payment Handled,2018/02/12 16:31:20


In [7]:
list_activities = list(tab_all["Activity"].unique())

lines, lines_t, lines_t2, lines_t3, lines_t4 = Extract_trace_and_temporal_features(tab_all)
maxlen = max([len(x) for x in lines]) #find maximum line size
lines, lines_t, lines_t2, lines_t3, lines_t4 = Extract_trace_and_temporal_features(tab_train)
divisor = np.mean([item for sublist in lines_t for item in sublist]) #average time between events
print('divisor: {}'.format(divisor))
divisor2 = np.mean([item for sublist in lines_t2 for item in sublist]) #average time between current and first events
print('divisor2: {}'.format(divisor2))
prefixes, outputs = Extract_prefix(lines, lines_t, lines_t2, lines_t3, lines_t4)
divisor_rt = np.mean(outputs[2])
print('divisor_rt: {}'.format(divisor_rt))

divisor: 186876.67318961537
divisor2: 500163.77877614804
divisor_rt: 623219.332072227


In [8]:
#creating instance of one-hot-encoder and fit on the whole dataset
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(np.array(list_activities).reshape((len(list_activities), 1)))

OneHotEncoder(handle_unknown='ignore')

In [9]:
def Prepare_X_Y_remaining_time(tab, list_activities, encoder, divisor, divisor2, divisor_rt):
    lines, lines_t, lines_t2, lines_t3, lines_t4 = Extract_trace_and_temporal_features(tab)
    prefixes, outputs = Extract_prefix(lines, lines_t, lines_t2, lines_t3, lines_t4)
    num_samples = len(prefixes[0])
    list_features = []
    list_edge_idx = []
    list_edge_weight = []
    list_rt = []
    for i, sentence in enumerate(prefixes[0]):
        list_rt.append(outputs[2][i]/divisor_rt)
        x = torch.zeros(len(sentence), len(list_activities)+5)
        edge_weight = []
        edge_idx = []
        sentence_t = prefixes[1][i]
        sentence_t2 = prefixes[2][i]
        sentence_t3 = prefixes[3][i]
        sentence_t4 = prefixes[4][i]
        for j, char in enumerate(sentence):
            x[j, :len(list_activities)] = torch.from_numpy(encoder.transform(np.array([[char]])).toarray()[0]) 
            x[j, len(list_activities)] = j+1
            x[j, len(list_activities)+1] = sentence_t[j]/divisor
            x[j, len(list_activities)+2] = sentence_t2[j]/divisor2
            x[j, len(list_activities)+3] = sentence_t3[j]/86400
            x[j, len(list_activities)+4] = sentence_t4[j]/7
            if j < len(sentence)-1:
                if sentence[j] == sentence[j+1]:
                    weight = 0
                elif sentence[j+1] in sentence[:j+1]:
                    weight = -1
                else:
                    weight = 1
                edge_weight.append(weight)
                edge_idx.append([j, j+1])
        list_features.append(x)
        list_edge_idx.append(torch.tensor(edge_idx).t().to(torch.long))
        list_edge_weight.append(torch.tensor(edge_weight).t())
    return [list_features, list_edge_idx, list_edge_weight], torch.tensor(list_rt)

In [10]:
X_train, Y_train = Prepare_X_Y_remaining_time(tab_train, list_activities, encoder, divisor, divisor2, divisor_rt)
X_valid, Y_valid = Prepare_X_Y_remaining_time(tab_valid, list_activities, encoder, divisor, divisor2, divisor_rt)
X_test, Y_test = Prepare_X_Y_remaining_time(tab_test, list_activities, encoder, divisor, divisor2, divisor_rt)

In [ ]:
#to save it
with open(data_dir+"GGNN_"+data_name+"_train.pkl", "wb") as f:
    pkl.dump([X_train, Y_train], f)
with open(data_dir+"GGNN_"+data_name+"_valid.pkl", "wb") as f:
    pkl.dump([X_valid, Y_valid], f)
with open(data_dir+"GGNN_"+data_name+"_test.pkl", "wb") as f:
    pkl.dump([X_test, Y_test], f)